In [ ]:
nb_pts = 5
hT = 1/nb_pts
xLin = [i*hT for i in range(1,nb_pts)]

In [ ]:
%%capture capt
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 12))

In [ ]:
import json
M = dict([('Map',{'Outer_dimension':3,'Arguments':[None]*4,'Arguments_metric':[None]*4}),('Dimension',2),('Cells',[[],[],[]])])
xDisp = []
yDisp = []
def add_vertex_multiple(X,Id):
    for i in range(len(X)):
        ax.text(X[i][0]+0.01,X[i][1]+0.01,'{}'.format(len(M['Cells'][0])),color='darkred',fontsize='small')
        xDisp.append(X[i][0])
        yDisp.append(X[i][1])
    M['Cells'][0].append({'Map_ids':Id,'Location':X})       
    
def add_vertex(x,y,map_id):
    ax.text(x+0.01,y+0.01,'{}'.format(len(M['Cells'][0])),color='red',fontsize='small')
    xDisp.append(x)
    yDisp.append(y)
    M['Cells'][0].append({'Map_ids':[map_id],'Location':[[x,y]]})



In [ ]:
add_vertex_multiple([[0,0],[1,0],[1,1],[0,1]],[0,1,2,3])
add_vertex_multiple([[xLin[0],0],[xLin[0],1],[xLin[0],1]],[0,2,3])
for i in range(1,len(xLin)-1):
    add_vertex_multiple([[xLin[i],0],[xLin[i],1]],[0,2])
add_vertex_multiple([[xLin[-1],0],[xLin[-1],0],[xLin[-1],1]],[0,1,2])
for i in range(len(xLin)-1):
    add_vertex_multiple([[0,xLin[i]],[1,xLin[i]]],[0,1])
add_vertex_multiple([[0,xLin[-1]],[0,xLin[-1]],[1,xLin[-1]],[1,xLin[-1]]],[0,3,1,2])

for i in range(len(xLin)-1):
    add_vertex(xLin[i],xLin[0],0)
for i in range(1,len(xLin)-1):
    add_vertex(xLin[0],xLin[i],0)
for i in range(len(xLin)-1):
    add_vertex_multiple([[xLin[-1],xLin[i]],[xLin[-1],xLin[i]]],[0,1]) 
for i in range(1,len(xLin)-1):
    add_vertex_multiple([[xLin[i],xLin[-1]],[xLin[i],xLin[-1]]],[0,2])
add_vertex_multiple([[xLin[-1],xLin[-1]],[xLin[-1],xLin[-1]],[xLin[-1],xLin[-1]]],[0,1,2])
add_vertex_multiple([[xLin[0],xLin[-1]],[xLin[0],xLin[-1]],[xLin[0],xLin[-1]]],[0,2,3])

for i in range(1,len(xLin)-1):
    for j in range(1,len(xLin)-1):
        add_vertex(xLin[j],xLin[i],0)
ax.scatter(xDisp,yDisp)

In [ ]:
def dispMap(mid):
    xDisp = [-0.1,-0.1,1.1,1.1]
    yDisp = [-0.1,1.1,-0.1,1.1]
    for V in M['Cells'][0]:
        if mid in V['Map_ids']:
            xDisp.append(V['Location'][V['Map_ids'].index(mid)][0])
            yDisp.append(V['Location'][V['Map_ids'].index(mid)][1])
    plt.scatter(xDisp,yDisp)
fig

In [ ]:
CellEdges = []
for i in range(nb_pts**2):
    CellEdges.append([])
def add_edge(v0,v1,Id,cIds):
    locV0 = M['Cells'][0][v0]['Location'][M['Cells'][0][v0]['Map_ids'].index(Id)]
    locV1 = M['Cells'][0][v1]['Location'][M['Cells'][0][v1]['Map_ids'].index(Id)]
    ax.text((locV0[0]+locV1[0])/2.,(locV0[1]+locV1[1])/2.,'{}'.format(len(M['Cells'][1])),fontsize='small')
    M['Cells'][1].append({'Map_ids':[Id],'Mappings':[0],'Ref_elem':None,'Boundary':[v0,v1]})
    for cid in cIds:
        (CellEdges[cid]).append(len(M['Cells'][1])-1)

def add_edge_multiple(v0,v1,Id,cIds):
    extraArg = []
    locV1 = []
    for mid in Id:
        locV0 = M['Cells'][0][v0]['Location'][M['Cells'][0][v0]['Map_ids'].index(mid)]
        locV1 = M['Cells'][0][v1]['Location'][M['Cells'][0][v1]['Map_ids'].index(mid)]
        extraArg.append([locV0[0],locV0[1],locV1[0],locV1[1]])
        ax.text((locV0[0]+locV1[0])/2.,(locV0[1]+locV1[1])/2.,'{}'.format(len(M['Cells'][1])),color='grey',fontsize='small')
    M['Cells'][1].append({'Map_ids':Id.copy(),'Mappings':[1]*len(Id),
                          'Mappings_extra_args': extraArg.copy(),
                          'Ref_elem':[[[0.],[1.]]],'Boundary':[v0,v1]})
    for cid in cIds:
        (CellEdges[cid]).append(len(M['Cells'][1])-1)



In [ ]:
offsetInner = 6*nb_pts-9
def innerSLoc(l,c):
    return offsetInner + (nb_pts-3)*l + c
def innerCellLoc(l,c):
    return 8*nb_pts - 16 + (nb_pts-4)*l + c

add_edge_multiple(0,1,[0,3],[0,3*nb_pts-3]) #Right 
add_edge_multiple(0,nb_pts-1,[1,2],[nb_pts-1,2*nb_pts-2]) #Left
add_edge_multiple(0,nb_pts,[0,1],[0,nb_pts-1]) #Up
add_edge_multiple(0,2*nb_pts-2,[2,3],[2*nb_pts-2,3*nb_pts-3]) #Down
for i in range(1,nb_pts-1): #Horizontal
    add_edge_multiple(i,i+1,[0,2],[i,3*nb_pts-3-i])
for i in range(nb_pts-2): #Vertical
    add_edge_multiple(nb_pts+i,nb_pts+i+1,[0,1],[nb_pts+i,4*nb_pts-5-i])
# Outside inner
for i in range(nb_pts-3): #B
    if (i < nb_pts - 4):
        add_edge(2*nb_pts+i,2*nb_pts+i+1,0,[2+i,4*nb_pts-3+i]) #H
    add_edge(2*nb_pts+i,2+i,0,[1+i,2+i]) #VO
    add_edge(2*nb_pts+i,offsetInner+i,0,[4*nb_pts-4+i,4*nb_pts-3+i]) #VI
for i in range(nb_pts-3): #L
    if (i < nb_pts - 4):
        add_edge(3*nb_pts-3+i,3*nb_pts-3+i+1,0,[4*nb_pts-6-i,8*nb_pts-17-i]) #H
    add_edge(3*nb_pts-3+i,nb_pts+1+i,0,[4*nb_pts-5-i,4*nb_pts-5-i-1]) #VO
    if (i==0):
        add_edge(3*nb_pts-3+i,offsetInner+i*(nb_pts-3),0,[4*nb_pts-4,8*nb_pts-17]) #VI
    else:
        add_edge(3*nb_pts-3+i,offsetInner+i*(nb_pts-3),0,[8*nb_pts-16-i,8*nb_pts-16-i-1]) #VI
for i in range(nb_pts-3): #R
    if (i < nb_pts - 4):
        add_edge_multiple(4*nb_pts-5+i,4*nb_pts-5+i+1,[0,1],[5*nb_pts-7+1+i,nb_pts+1+i]) #H
    add_edge(4*nb_pts-5+i,nb_pts+1+i,1,[nb_pts+i,nb_pts+i+1]) #VO
    add_edge(4*nb_pts-5+i,7*nb_pts-13+i*(nb_pts-3),0,[5*nb_pts-7+i,5*nb_pts-7+1+i]) #VI
for i in range(nb_pts-3): #T
    if (i < nb_pts - 4):
        add_edge_multiple(5*nb_pts-8+i,5*nb_pts-8+i+1,[0,2],[7*nb_pts-14-i,3*nb_pts-5-i]) #H
    add_edge(5*nb_pts-8+i,2+i,2,[3*nb_pts-4-i,3*nb_pts-5-i]) #VO
    add_edge(5*nb_pts-8+i,offsetInner+(nb_pts-3)**2-(nb_pts-3)+i,0,[7*nb_pts-13-i,7*nb_pts-14-i]) #VI
# Diagonal
# BL
add_edge(2*nb_pts-1,2*nb_pts,0,[1,4*nb_pts-4])
add_edge(2*nb_pts-1,1,0,[0,1])
add_edge(2*nb_pts-1,nb_pts,0,[0,4*nb_pts-5])
add_edge(2*nb_pts-1,3*nb_pts-3,0,[4*nb_pts-5,4*nb_pts-4])
# BR
add_edge(4*nb_pts-6,nb_pts,1,[nb_pts-1,nb_pts])
add_edge_multiple(4*nb_pts-6,nb_pts-1,[0,1],[nb_pts-2,nb_pts-1])
add_edge(4*nb_pts-6,3*nb_pts-4,0,[nb_pts-2,5*nb_pts-7])
add_edge_multiple(4*nb_pts-6,4*nb_pts-5,[0,1],[5*nb_pts-7,nb_pts])
# TR
add_edge_multiple(6*nb_pts-11,2*nb_pts-2,[1,2],[2*nb_pts-3,2*nb_pts-2])
add_edge_multiple(6*nb_pts-11,5*nb_pts-9,[0,1],[6*nb_pts-10,2*nb_pts-3])
add_edge_multiple(6*nb_pts-11,6*nb_pts-12,[0,2],[6*nb_pts-10,2*nb_pts-1])
add_edge(6*nb_pts-11,nb_pts-1,2,[2*nb_pts-1,2*nb_pts-2])
# TL
add_edge_multiple(6*nb_pts-10,5*nb_pts-8,[0,2],[7*nb_pts-13,3*nb_pts-4])
add_edge(6*nb_pts-10,4*nb_pts-7,0,[7*nb_pts-13,3*nb_pts-2])
add_edge_multiple(6*nb_pts-10,2*nb_pts-2,[0,3],[3*nb_pts-2,3*nb_pts-3])
add_edge_multiple(6*nb_pts-10,1,[2,3],[3*nb_pts-3,3*nb_pts-4])
# Inner square
for c in range(nb_pts-4):
    add_edge(innerSLoc(0,c),innerSLoc(0,c+1),0,[4*nb_pts-3+c,innerCellLoc(0,c)])
    add_edge(innerSLoc(nb_pts-4,c),innerSLoc(nb_pts-4,c+1),0,[7*nb_pts-14-c,innerCellLoc(nb_pts-5,c)])
for l in range(nb_pts-4):
    add_edge(innerSLoc(l,0),innerSLoc(l+1,0),0,[8*nb_pts-17-l,innerCellLoc(l,0)])
    add_edge(innerSLoc(l,nb_pts-4),innerSLoc(l+1,nb_pts-4),0,[5*nb_pts-6+l,innerCellLoc(l,nb_pts-5)])
for l in range(1,nb_pts-4):
    for c in range(nb_pts-4):
        add_edge(innerSLoc(l,c),innerSLoc(l,c+1),0,[innerCellLoc(l,c),innerCellLoc(l-1,c)])
for c in range(1,nb_pts-4):
    for l in range(nb_pts-4):
        add_edge(innerSLoc(l,c),innerSLoc(l+1,c),0,[innerCellLoc(l,c),innerCellLoc(l,c-1)])

In [ ]:
fig

In [ ]:
def add_face(map_id):
    edges = CellEdges[len(M['Cells'][2])]
    # plot
    xmid = 0.
    ymid = 0.
    for i in edges:
        for iV in M['Cells'][1][i]['Boundary']:
            xmid += M['Cells'][0][iV]['Location'][M['Cells'][0][iV]['Map_ids'].index(map_id)][0]
            ymid += M['Cells'][0][iV]['Location'][M['Cells'][0][iV]['Map_ids'].index(map_id)][1]
    xmid = xmid/(2.*len(edges))
    ymid = ymid/(2.*len(edges))
    ax.text(xmid,ymid,'{}'.format(len(M['Cells'][2])),color='blue',fontsize='small')
    # add data
    M['Cells'][2].append({'Map_ids':[map_id],'Mappings':[0],'Ref_elem':None,'Boundary':edges.copy()})

def add_face_nflat(map_id):
    edges = CellEdges[len(M['Cells'][2])]
    # plot
    xmid = 0.
    ymid = 0.
    for i in edges:
        for iV in M['Cells'][1][i]['Boundary']:
            xmid += M['Cells'][0][iV]['Location'][M['Cells'][0][iV]['Map_ids'].index(map_id)][0]
            ymid += M['Cells'][0][iV]['Location'][M['Cells'][0][iV]['Map_ids'].index(map_id)][1]
    xmid = xmid/(2.*len(edges))
    ymid = ymid/(2.*len(edges))
    ax.text(xmid,ymid,'{}'.format(len(M['Cells'][2])),color='darkblue',fontsize='small')
    # add data
    v1 = [-0.5,-0.5]
    v2 = [-0.5,0.5]
    v3 = [0.5,0.5]
    v4 = [0.5,-0.5]
    M['Cells'][2].append({'Map_ids':[map_id],'Mappings':[1],
                          'Mappings_extra_args':[[xmid,ymid,hT]],
                          'Ref_elem':[[v1,v2,v3],[v1,v3,v4]],
                          'Boundary':edges.copy()})


In [ ]:
for i in range(nb_pts-1):
    add_face_nflat(0)
for i in range(nb_pts-1,2*nb_pts-2):
    add_face_nflat(1)
for i in range(2*nb_pts-2,3*nb_pts-3):
    add_face_nflat(2)
add_face_nflat(3)
for i in range(3*nb_pts-2,4*nb_pts-4):
    add_face_nflat(0)
for i in range(4*nb_pts-4,5*nb_pts-7):
    add_face(0)
for i in range(5*nb_pts-7,7*nb_pts-12):
    add_face_nflat(0)
for i in range(7*nb_pts-12,nb_pts**2):
    add_face(0)

In [ ]:
fig

In [ ]:
with open('./torus_{}.json'.format(nb_pts),'w') as f:
    json.dump(M,f, indent = 1)